# Model Coding 기본 구조

* 구조 예) Linear Regression
* Regression Analysis(회귀분석)
  : 독립변수와 종속변수 사이에 인과관계가 존재할 때 그 관계의 통계적 유의미성을 검증하고, 
     또 그 관계의 정도를 분석하는 것

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

# Dataset은 아래 두 가지로 구성 가능

1. 입력 Data가 torch.FloatTensor 인 경우

2. 일반 행렬 Data를 pytorch의 Tensor로 변환해야 하는 경우

** import 함수가 다르며, 아래는 2번으로 진행되고, 1번은 Markdown 처리함)

In [ ]:
#1
from torch.utils.data import TensorDataset
x_trains = torch.FloatTensor([[73, 80, 75],
                              [93, 88, 93],
                              [89, 91, 90],
                              [96, 98, 100],
                              [73, 66, 70]])
y_trains = torch.FloatTensor([[152], [185], [180], [196], [142]])
dataset = TensorDataset(x_trains, y_trains)

In [11]:
#2
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

dataset = CustomDataset()

In [18]:
# CustomDataset 크기 확인
dataset.__len__()

5

In [19]:
# CustomDataset i번째 원소 확인
i=1
dataset.__getitem__(i)

(tensor([93., 88., 93.]), tensor([185.]))

In [6]:
dataloader = DataLoader (dataset, batch_size = 2, shuffle=True)

# Model 정의 방법

본 예시의 경우, model = nn.Linear(3,1)로 간단히 정의 가능하나, 향후 복잡한 문제 해결을 위해 class로 구현함

In [8]:
class MultiVariateLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)
        
    def forward(self, x):
        return self.linear(x)

model = MultiVariateLinear()

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

# Model 학습

In [11]:
nb_epochs = 200
for epoch in range(nb_epochs +1):
    for batch_idx, samples in enumerate(dataloader):
#        print(batch_idx)     # enumerate 함수 작동을 보려면 주석 해제
#        print(samples)
        x_train, y_train = samples
        
        prediction = model(x_train)
        
        cost = F.mse_loss(prediction, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        if epoch % 50 == 0:
            print('Epoch {:4d}/{} Batch{}/{} Cost: {:.6f}'.format(
                epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
                ))

Epoch    0/200 Batch1/3 Cost: 1.094231
Epoch    0/200 Batch2/3 Cost: 0.155639
Epoch    0/200 Batch3/3 Cost: 1.430706
Epoch   50/200 Batch1/3 Cost: 0.898428
Epoch   50/200 Batch2/3 Cost: 0.558950
Epoch   50/200 Batch3/3 Cost: 1.512739
Epoch  100/200 Batch1/3 Cost: 0.335478
Epoch  100/200 Batch2/3 Cost: 1.821562
Epoch  100/200 Batch3/3 Cost: 0.025616
Epoch  150/200 Batch1/3 Cost: 0.329790
Epoch  150/200 Batch2/3 Cost: 1.483798
Epoch  150/200 Batch3/3 Cost: 0.897283
Epoch  200/200 Batch1/3 Cost: 0.109546
Epoch  200/200 Batch2/3 Cost: 0.907395
Epoch  200/200 Batch3/3 Cost: 0.979899


# 학습된 Model로 예측

In [12]:
new_var = torch.FloatTensor([[73, 80,75]])
pred_y = model(new_var)
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[152.2095]], grad_fn=<AddmmBackward>)


* Reference

  -. https://wikidocs.net/54841
  
  -. https://wikidocs.net/60036